# Create 3D SVR volume DICOM

Converts 3D Nifti file into single-frame Dicom files

INPUT:
- 3D SVR .nii file
- M2D Dicom for header information

OUTPUT:
- 3D SVR Dicom

### User Inputs

In [ ]:
dcmInPath = r'C:\Users\tr17\Dropbox\PrideSVR\Untouched_Dicoms_Exported_from_Simulator\DICOM\IM_0001'
niiInPath = r'C:\Users\tr17\Dropbox\PrideSVR\pride_export_complete\SVR-output-brain.nii.gz'

dcmOutPath = r''

In [ ]:
import os
import numpy as np
import numpy.matlib
import nibabel as nib
import matplotlib
import matplotlib.pyplot as plt
import pydicom as pyd

In [ ]:
dcmIn = pyd.dcmread(dcmInPath)
print('dcmIn')
print(dcmIn.SeriesNumber)

In [ ]:
# parse nifti
niiIn   = nib.load(niiInPath)
nii_img = niiIn.get_fdata()

if niiIn.header['dim'][4] == 1:
    nX, nY, nZ, nF   = niiIn.header['dim'][1], niiIn.header['dim'][2], niiIn.header['dim'][3], 1
    dimX, dimY, dimZ = niiIn.header['pixdim'][1], niiIn.header['pixdim'][2], niiIn.header['pixdim'][3]

    print("dim [nX, nY, nZ]:", [nX, nY, nZ])
    print("pixdim [mm, mm, mm]:", [dimX, dimY, dimZ])

elif niiIn.header['dim'][4] > 1:
    print("Warning: Nifti is not 3-D ...")

# number of instances
nInstances = nZ*nF

# set background pixels = 0 (-1 in SVRTK)
iBkrd = nii_img==-1; nii_img[iBkrd] = 0

# match DICOM datatype
nii_img = nii_img.astype("uint16")

# slice location arrays
sliceIndices = np.repeat(range(1, nZ+1), nF)

# slice locations array
voxelSpacing = dimZ
zLocLast = (voxelSpacing * nZ) - voxelSpacing
sliceLoca = np.repeat(np.linspace(0, zLocLast, num=nZ), nF)

# TODO: windowing
# windowCenter = []
# windowWidth = []
# rescaleIntercept = []
# rescaleSlope = []

In [ ]:
# nii parameters to transfer
nii_parameters = {
    'SliceThickness': str(nZ),
    'SpacingBetweenSlices': str(nZ),
    'AcquisitionMatrix': [0, nX, nY, 0],
    'InstanceNumber': sliceIndices,
    'SliceLocation': sliceLoca,
    'Rows': nX,
    'Columns': nY,
    'PixelSpacing': [nX, nY],
    'WindowCenter': str(1000),
    'WindowWidth': str(1800),
    'RescaleIntercept': str(0),
    'RescaleSlope': str(21),
}

# UIDs
# uid_instance_creator = pyd.uid.generate_uid(None) # think can transfer
uid_instance         = pyd.uid.generate_uid(None) # per slice
uid_series_instance  = pyd.uid.generate_uid(None) # per svr

# per SVR parameters to create
series_number = str(2) + str(dcmIn.SeriesNumber)
acquisition_number = str(2) + str(dcmIn.AcquisitionNumber)

# per slice parameters to create
instance_number = 1 # instance_number = loop over slice indices
slice_location = 1 # slice_location = loop over slice locations

In [ ]:
# file_meta
elements_to_define_meta = {
    'FileMetaInformationGroupLength': 210, # auto-generated?
    'MediaStorageSOPClassUID': '1.2.840.10008.5.1.4.1.1.4',
    'ImplementationVersionName': 'RESEARCH_SVR_DICOM',
    'MediaStorageSOPInstanceUID': uid_instance,
}

elements_to_transfer_meta = {
    'FileMetaInformationVersion': 'FileMetaInformationVersion',
    'TransferSyntaxUID': 'TransferSyntaxUID',
    'ImplementationClassUID': 'ImplementationClassUID',
}


# Dataset
elements_to_define_ds = {
    'SOPInstanceUID': uid_instance, # per slice
    'SliceThickness': nii_parameters['SliceThickness'], # per SVR
    'SpacingBetweenSlices': nii_parameters['SpacingBetweenSlices'], # per SVR
    'ProtocolName': 'SVR_PRIDE_RESEARCH_RECON',
    'AcquisitionMatrix': nii_parameters['AcquisitionMatrix'], # per SVR
    'SeriesInstanceUID': uid_series_instance,
    'SeriesNumber': series_number, # per SVR
    'AcquisitionNumber': acquisition_number, # per SVR
    'InstanceNumber': instance_number, # per slice
    'ImageOrientationPatient': ['1','0','0','0','1','0'],
    # 'ImagePositionPatient': [x,x,x], # omitted
    # 'ImageOrientationPatient': [x,x,x,x,x,x], # omitted
    # 'FrameOfReferenceUID': uid_frame_of_reference # omit, define or transfer?
    'TemporalPositionIdentifier': '1',
    'NumberOfTemporalPositions': '1',
    'SliceLocation': slice_location,
    'SamplesPerPixel': 1,
    'WindowCenter': nii_parameters['WindowCenter'],
    'WindowWidth': nii_parameters['WindowWidth'],
    'RescaleIntercept': nii_parameters['RescaleIntercept'],
    'RescaleSlope': nii_parameters['RescaleSlope'],
}

elements_to_transfer_ds = {    
    # general
    'SpecificCharacterSet': 'SpecificCharacterSet',
    'ImageType': 'ImageType',
    'InstanceCreationDate': 'InstanceCreationDate', # match 2D dcm ?
    'InstanceCreationTime': 'InstanceCreationTime',
    'InstanceCreatorUID': 'InstanceCreatorUID',
    'SOPClassUID': 'SOPClassUID',
    'StudyDate': 'StudyDate',
    'SeriesDate': 'StudyDate',
    'AcquisitionDate': 'AcquisitionDate',
    'ContentDate': 'ContentDate',
    'StudyTime': 'StudyTime',
    # 'SeriesTime': 'SeriesTime', # define?
    # 'AcquisitionTime': 'AcquisitionTime', # define?
    # 'ContentTime': 'ContentTime', # define? --- nb: same as AcquisitionTime
    'AccessionNumber': 'AccessionNumber',
    'Modality': 'Modality',
    'ConversionType': 'ConversionType',
    'Manufacturer': 'Manufacturer',
    'InstitutionName': 'InstitutionName',
    'InstitutionAddress': 'InstitutionAddress',
    'ReferringPhysicianName': 'ReferringPhysicianName',
    'CodeValue': 'CodeValue',
    'CodingSchemeDesignator': 'CodingSchemeDesignator',
    'CodeMeaning': 'CodeMeaning',
    'StationName': 'StationName',
    'StudyDescription': 'StudyDescription',
    'InstitutionalDepartmentName': 'InstitutionalDepartmentName',
    'PerformingPhysicianName': 'PerformingPhysicianName',
    'OperatorsName': 'OperatorsName',
    'ManufacturerModelName': 'ManufacturerModelName',
    
    # patient
    'PatientName': 'PatientName',
    'PatientID': 'PatientID',
    'PatientBirthDate': 'PatientBirthDate',
    'PatientSex': 'PatientSex',
    'PatientAge': 'PatientAge',
    'PatientWeight': 'PatientWeight',
    'PregnancyStatus': 'PregnancyStatus',
    'BodyPartExamined': 'BodyPartExamined',
    
    # series
    'ScanningSequence': 'ScanningSequence',
    'SequenceVariant': 'SequenceVariant',
    'ScanOptions': 'ScanOptions',
    'MRAcquisitionType': 'MRAcquisitionType', # = '2D' in original dicom - problematic? Set '3D' or blank?
    'RepetitionTime': 'RepetitionTime',
    'EchoTime': 'EchoTime',
    'NumberOfAverages': 'NumberOfAverages',
    'ImagingFrequency': 'ImagingFrequency',
    'ImagedNucleus': 'ImagedNucleus',
    'EchoNumbers': 'EchoNumbers',
    'MagneticFieldStrength': 'MagneticFieldStrength',
    'NumberOfPhaseEncodingSteps': '', # only relevant to 2D series - make blank or transfer?
    'EchoTrainLength': '', # only relevant to 2D series - make blank or transfer?
    'PercentSampling': '', # only relevant to 2D series - make blank or transfer?
    'PercentPhaseFieldOfView': '', # only relevant to 2D series - make blank or transfer?
    'PixelBandwidth': '', # only relevant to 2D series - make blank or transfer?
    'DeviceSerialNumber': 'DeviceSerialNumber',
    # ... skipped some SecondardCaptureDevice fields
    'SoftwareVersions': 'SoftwareVersions',
    'TriggerTime': 'TriggerTime',
    'LowRRValue': 'LowRRValue',
    'HighRRValue': 'HighRRValue',
    'IntervalsAcquired': 'IntervalsAcquired',
    'IntervalsRejected': 'IntervalsRejected',
    'HeartRate': 'HeartRate',
    'TriggerWindow': 'TriggerWindow',
    'ReconstructionDiameter': '', # only relevant to 2D series - make blank or transfer?
    'ReceiveCoilName': 'ReceiveCoilName',
    'TransmitCoilName': 'TransmitCoilName',
    'InPlanePhaseEncodingDirection': '', # only relevant to 2D series - make blank or transfer?
    'FlipAngle': 'FlipAngle',
    'SAR': '', # only relevant to 2D series - make blank or transfer?
    'dBdt': '', # only relevant to 2D series - make blank or transfer?
    'B1rms': '', # only relevant to 2D series - make blank or transfer?
    'PatientPosition': 'PatientPosition',
    'AcquisitionDuration': '', # only relevant to 2D series - make blank or transfer?
    'DiffusionBValue': 'DiffusionBValue',
    'DiffusionGradientOrientation': 'DiffusionGradientOrientation',
    'StudyInstanceUID': 'StudyInstanceUID',
    'StudyID': 'StudyID',
    'PhotometricInterpretation': 'PhotometricInterpretation',
    'BitsAllocated': 'BitsAllocated',
    'BitsStored': 'BitsStored',
    'HighBit': 'HighBit',
    'PixelRepresentation': 'PixelRepresentation',
    'LossyImageCompression': 'LossyImageCompression',
    'RequestingPhysician': 'RequestingPhysician',
    'RequestingService': 'RequestingService',
    'RequestedProcedureDescription': 'RequestedProcedureDescription',
    'RequestedContrastAgent': 'RequestedContrastAgent',
    'PerformedStationAETitle': 'PerformedStationAETitle',
    'PerformedStationName': 'PerformedStationName',
    'PerformedLocation': 'PerformedLocation',
    
    # performed procedure step
    # # required ?
    # ds.PerformedProcedureStepStartDate = '20200101'
    # ds.PerformedProcedureStepStartTime = '133010'
    # ds.PerformedProcedureStepEndDate = '20200101'
    # ds.PerformedProcedureStepEndTime = '133010'
    # ds.PerformedProcedureStepStatus = ''
    # ds.PerformedProcedureStepID = '631715410'
    # ds.PerformedProcedureStepDescription = ''
    # ds.PerformedProcedureTypeDescription = ''
    
    ### fields not transferred
    # Procedure Code Sequence
    # Referenced Study Sequence
    # Referenced Performed Procedure Step Sequence
    # Performed Protocol Code Sequence
    # Request Attributes Sequence
    # Scheduled Protocol Code Sequence
    # ... some more general fields
    # Real World Value Mapping Sequence
    # Measurement Units Code Sequence
    # ... possibly others

    '': '',  
}


In [ ]:
def dcm_initialise():

    # file_meta
    file_meta = pyd.Dataset()

    for k, v in elements_to_define_meta.items():
        setattr(file_meta, k, v)

    for k, v in elements_to_transfer_meta.items():
        try:
            setattr(file_meta, k, getattr(dcmIn.file_meta, v))
        except:
            log.warning(f"Could not transfer tag for keyword {k}")

    # dataset
    ds = pyd.Dataset()
    ds.file_meta = file_meta
    ds.is_implicit_VR = False
    ds.is_little_endian = True

    for k, v in elements_to_define_ds.items():
        setattr(ds, k, v)

    for k, v in elements_to_transfer_ds.items():
        try:
            setattr(ds, k, getattr(dcmIn, v))
        except:
            log.warning(f"Could not transfer tag for keyword {k}")


In [ ]:
# create dicoms

iFileCtr = 1

for iImage in range(0,nInstances):

    file_meta, ds = dcm_initialise()

    ds.PixelData = nii_img[:,:,iImage,0].tobytes()
    ds.save_as( os.path.join(dcmOutPath, r'IM_%04d'%(iFileCtr), write_like_original=False ) )
    iFileCtr = iFileCtr + 1

# Output Messages
print('SVR 3D DICOM creation complete.')
print('Output directory:', os.path.join(dcmOutPath) )
